In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#link do folderu, w którym mamy bazę:

%cd "C:\Users\Dawid\Desktop\kurs\projekt\uber"

C:\Users\Dawid\Desktop\kurs\projekt\uber


In [3]:
occupants=pd.read_csv("./occupants.csv")
accidents=pd.read_csv("./accidents.csv")
vehicles=pd.read_csv("./vehicles.csv")

In [6]:
vehicles.loc[(vehicles["BODYTYPE"]==5) | (vehicles["BODYTYPE"]==0)]

,CASEID,PSU,YEAR,BODYTYPE,MAKE,DVTOTAL,MODELYR
0,1H,2,2004,0.0,37.0,NaN,1995.0
1,2K,2,2004,5.0,37.0,37.0,2001.0
2,2K,2,2004,5.0,49.0,40.0,1998.0
3,3E,2,2004,0.0,20.0,15.0,2001.0
4,3E,2,2004,0.0,55.0,22.0,2001.0
...,...,...,...,...,...,...,...
94593,72H,82,2015,0.0,37.0,NaN,2010.0
94594,72H,82,2015,0.0,48.0,NaN,2013.0
94595,72H,82,2015,0.0,41.0,NaN,2007.0
94596,76H,82,2015,0.0,42.0,NaN,1990.0


In [4]:
# 1. dodanie kolumny z wiekem samochodu
vehicles["CarAge"] = np.where((vehicles.YEAR - vehicles.MODELYR) < 0, 0,vehicles.YEAR - vehicles.MODELYR)

In [5]:
# 2. usuniecie innych kategorii samochodu niz osobowe
vehicles=vehicles[vehicles["BODYTYPE"].isin([0,5])]

In [6]:
# 3. Dodajemy informacje o pasażerach w tym samochodzie
tabela=vehicles.merge(occupants, on = ["CASEID","PSU","YEAR"])

In [7]:
# 4.odfiltrowujemy nieokreślone obrażenia
tabela = tabela[tabela["INJSEV"]!=5]
# 5. grupuje obrażenia:

przypisanie = [(tabela.INJSEV == 0),
               (tabela.INJSEV == 1) | (tabela.INJSEV == 2),
               (tabela.INJSEV == 3),
               (tabela.INJSEV == 4) | (tabela.INJSEV == 6)]

co_przypisac = [0,1,3,4]

tabela["INJSEV"] = np.select(przypisanie, co_przypisac)

# 0: brak obrażen
# 1: odniesione lekkie obrażenia (1,2)
# 3: odniesione poważne obrażenia (3)
# 4: śmierć (4,6)



In [8]:
# 6 usunięcie NA z wieku samochodu
tab0=tabela.dropna(subset=["MODELYR"])



In [9]:
# 7 pogrupowanie wieku samochodu
warunek = [
    (tab0.CarAge <= 2),
    (tab0.CarAge <= 5) & (tab0.CarAge >2),
    (tab0.CarAge <= 10) & (tab0.CarAge >5),
    (tab0.CarAge <= 20) & (tab0.CarAge >10),
    (tab0.CarAge <= 30) & (tab0.CarAge >20),
    (tab0.CarAge > 30)
]

wartosc = range(6)

tab0["CarAge"] = np.select(warunek, wartosc)



# 0 - 2 lata (0)
# 3 - 5 lata (1)
# 6 - 10 lata (2)
# 11-20 lat (3)
# 21 -30 lat (4)
# 31 lat i więcej (5)


C:\Users\Dawid\anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


**ANALIZY GLOWNA**

In [10]:
#1 wpływ wieku samochodu na skalę obrażeń
#usuwam wartosci NA z kolumny obrazenia
tab=tab0.dropna(subset=["INJSEV"])

#wybieram tylko kolumny, ktore potrzebuje i robie 2 grupowania, aby wyliczyc skale obrazen w ramach danego wieku auta (udzial)
pom1=tab.groupby(["CarAge"])["PSU"].count().to_frame().reset_index()
pom2=tab.groupby(["CarAge","INJSEV"])["PSU"].count().to_frame().reset_index()

In [11]:
# zmieniam nazwe kolumn
pom2.rename(columns={'PSU':'NoAccidents'},inplace = True)
pom1.rename(columns={"PSU":'NoAccPerGroup'}, inplace = True )

In [12]:
# polacze obie pomocnicze tabele i policze wspolczynnik
zbiorcza=pom2.merge(pom1, on = "CarAge")
zbiorcza["ratio"] = zbiorcza.NoAccidents/zbiorcza.NoAccPerGroup
CarAgeResult = zbiorcza.groupby(["CarAge","INJSEV"])["ratio"].mean().to_frame().reset_index()

Mamy policzone prawdododobienstwo danego obrazenia przy wypadku ze wzgledu na wiek auta.  
Teraz w kolejnym kroku prawdopodobienstwo danego obrazenia przy wypadku ze względu na markę auta


In [13]:
#2 wpływ marki samochodu na skalę obrażeń
#usuwam wartosci NA z kolumny MAKE (marka)
tab2=tabela.dropna(subset=["MAKE"])

#dodaje marki, ktore wystapily mniej niz 100 razy do grupy inne, tj. 99 : 

tab2["MAKE"]=np.where(tab2["MAKE"].isin([31,84,50,43,60,33,29,1,82,25,36,67,65,10,69,3,14,18,23,24,54,58,9,21]),99, tab2["MAKE"])

#wybieram tylko kolumny, ktore potrzebuje i robie 2 grupowania, aby wyliczyc skale obrazen w ramach danej marki auta (udzial)
pom11=tab2.groupby(["MAKE"])["PSU"].count().to_frame().reset_index()
pom22=tab2.groupby(["MAKE","INJSEV"])["PSU"].count().to_frame().reset_index()

In [14]:
# zmieniam nazwe kolumn
pom22.rename(columns={'PSU':'NoAccidents'},inplace = True)
pom11.rename(columns={"PSU":'NoAccPerGroup'}, inplace = True )

In [15]:
# polacze obie pomocnicze tabele i policze wspolczynnik
zbiorcza2=pom22.merge(pom11, on = "MAKE")
zbiorcza2["ratio"] = zbiorcza2.NoAccidents/zbiorcza2.NoAccPerGroup
MakeResult= zbiorcza2.groupby(["MAKE","INJSEV"])["ratio"].mean().to_frame().reset_index()

W kolejnym kroku zbadamy wpływ pozycji siedzenia pasażera na skale obrażeń przy ewentualnym wypadku.

In [16]:
#3 wpływ pozycji siedzenia na skalę obrażeń
#usuwam wartosci NA z kolumny SEATPOS (marka)
tab3=tabela.dropna(subset=["SEATPOS"])

#wybieram tylko kolumny, ktore potrzebuje i robie 2 grupowania, aby wyliczyc skale obrazen ze wzgledu na pozycje (udzial)
pom111=tab3.groupby(["SEATPOS"])["PSU"].count().to_frame().reset_index()
pom222=tab3.groupby(["SEATPOS","INJSEV"])["PSU"].count().to_frame().reset_index()

In [17]:
# zmieniam nazwe kolumn
pom222.rename(columns={'PSU':'NoAccidents'},inplace = True)
pom111.rename(columns={"PSU":'NoAccPerGroup'}, inplace = True )

In [18]:
# polacze obie pomocnicze tabele i policze wspolczynnik
zbiorcza3=pom222.merge(pom111, on = "SEATPOS")
zbiorcza3["ratio"] = zbiorcza3.NoAccidents/zbiorcza3.NoAccPerGroup
SeatposResult = zbiorcza3.groupby(["SEATPOS","INJSEV"])["ratio"].mean().to_frame().reset_index()

W następnej części zbadamy wpływ wieku kierowcy.


In [19]:
#4 wpływ wieku kierowcy na skalę obrażeń
# najpierw wyszczegolnie tylko kierowcow
drivers = occupants.loc[occupants["ROLE"]==1,["CASEID","PSU","YEAR","ROLE", "AGE"]]

#usuwam wartosci NA z kolumny AGE (wiek dla kierowcow)
AgePerCase= drivers.dropna(subset=["AGE"])

#wezme najpierw tylko pierwszego kierowce z wypadkow w ktorych bylo wiecej aut
AgePerCase_1= AgePerCase[AgePerCase.duplicated(keep= "first",subset=["CASEID","PSU","YEAR"])]
AgePerCase_1= AgePerCase[["CASEID","PSU","YEAR","AGE"]]

#dla wypadkow,ktore sie nie dublowaly biore wszystko
AgePerCase_2= AgePerCase[AgePerCase.duplicated(keep= False,subset=["CASEID","PSU","YEAR"])]
AgePerCase_2= AgePerCase[["CASEID","PSU","YEAR","AGE"]]

AgePerCase = AgePerCase_1.append(AgePerCase_2)

#przypisze do danego wypadku wiek kierowcy
passagers = tabela.loc[tabela["ROLE"]==2,["CASEID","PSU","YEAR","ROLE","INJSEV"]].drop_duplicates()
tab4 = passagers.merge(AgePerCase, on = ["CASEID","PSU","YEAR"], how ='left').drop_duplicates().dropna()
tab4.rename(columns = {"AGE": "DriverAge"},inplace = True)

#usunę przypadki, ktore wystapily marginalnie, czy gdy mial mniej niz 16 lat
tab4 = tab4[(tab4["DriverAge"]>3) & (tab4["DriverAge"]<9)]


In [20]:
#wybieram tylko kolumny, ktore potrzebuje i robie 2 grupowania, aby wyliczyc skale obrazen w ramach wieku kierowcy danego wypadku (udzial)
pom1111=tab4.groupby(["DriverAge"])["PSU"].count().to_frame().reset_index()
pom2222=tab4.groupby(["DriverAge","INJSEV"])["PSU"].count().to_frame().reset_index()

In [21]:
# zmieniam nazwe kolumn
pom2222.rename(columns={'PSU':'NoAccidents'},inplace = True)
pom1111.rename(columns={"PSU":'NoAccPerGroup'}, inplace = True )

In [22]:
# polacze obie pomocnicze tabele i policze wspolczynnik
zbiorcza4=pom2222.merge(pom1111, on = "DriverAge")
zbiorcza4["ratio"] = zbiorcza4.NoAccidents/zbiorcza4.NoAccPerGroup
DriverAgeResult = zbiorcza4.groupby(["DriverAge","INJSEV"])["ratio"].mean().to_frame().reset_index()

W tej części zbadamy wpływ płci kierowcy.


In [23]:
#5 wpływ płci kierowcy na skalę obrażeń
# najpierw wyszczegolnie tylko kierowcow
drivers1 = occupants.loc[occupants["ROLE"]==1,["CASEID","PSU","YEAR","ROLE", "SEX"]]

#usuwam wartosci NA z kolumny SEX (wiek dla kierowcow)
SexPerCase = drivers1.dropna(subset=["SEX"])

#wezme najpierw tylko pierwszego kierowce z wypadkow w ktorych bylo wiecej aut
SexPerCase_1= SexPerCase[SexPerCase.duplicated(keep = "first",subset=["CASEID","PSU","YEAR"])]
SexPerCase_1= SexPerCase[["CASEID","PSU","YEAR","SEX"]]

#dla wypadkow,ktore sie nie dublowaly biore wszystko
SexPerCase_2= SexPerCase[SexPerCase.duplicated(keep= False,subset=["CASEID","PSU","YEAR"])]
SexPerCase_2= SexPerCase[["CASEID","PSU","YEAR","SEX"]]

SexPerCase = SexPerCase_1.append(SexPerCase_2)

#przypisze do danego wypadku płec kierowcy
passagers1 = tabela.loc[tabela["ROLE"]==2,["CASEID","PSU","YEAR","ROLE","INJSEV"]].drop_duplicates()
tab5 = passagers.merge(SexPerCase, on = ["CASEID","PSU","YEAR"], how ='left').drop_duplicates().dropna()
tab5.rename(columns = {"SEX": "DriverSex"},inplace = True)

#pogrupuje kobiete w ciąży jako kobiete

tab5["DriverSex"] = np.where(tab5["DriverSex"]==3,2,tab5["DriverSex"])

In [24]:
#wybieram tylko kolumny, ktore potrzebuje i robie 2 grupowania, aby wyliczyc skale obrazen w ramach płci kierowcy z danego wypadku (udzial)
pom11111=tab5.groupby(["DriverSex"])["PSU"].count().to_frame().reset_index()
pom22222=tab5.groupby(["DriverSex","INJSEV"])["PSU"].count().to_frame().reset_index()


In [25]:
# zmieniam nazwe kolumn
pom22222.rename(columns={'PSU':'NoAccidents'},inplace = True)
pom11111.rename(columns={"PSU":'NoAccPerGroup'}, inplace = True )

In [26]:
# polacze obie pomocnicze tabele i policze wspolczynnik
zbiorcza5=pom22222.merge(pom11111, on = "DriverSex")
zbiorcza5["ratio"] = zbiorcza5.NoAccidents/zbiorcza5.NoAccPerGroup
DriverSexResult = zbiorcza5.groupby(["DriverSex","INJSEV"])["ratio"].mean().to_frame().reset_index()

**BADAMY TERAZ ŁĄCZNE PRAWDOPODOBIENSTWO PO PODANIU KRYTERIÓW**

In [27]:
# 1. lista, ankieta:
## wiek auta z przedziału
CarAge = 3

# 0: 0 - 2 lata 
# 1: 3 - 5 lata 
# 2: 6 - 10 lata 
# 3: 11-20 lat
# 4: 21 -30 lat 
# 5: 31 lat i więcej 

## marka auta z listy poniżej
MAKE = 7

# 2 Jeep
# 6 Chrysler
# 7 Dodge
# 12 Ford
# 13 Lincoln
# 19 Cadillac
# 20 Chevrolet
# 22 Pontiac
# 30 Volkswagen
# 32 Audi
# 34 Bmw
# 35 Nissan/Datsun
# 37 Honda
# 38 Isuzu
# 42 Mercedes Benz
# 45 Porsche
# 47 Saab
# 48 Subaru
# 49 Toyota
# 51 Volvo
# 52 Mitsubishi
# 53 Suzuki
# 55 Hyundai
# 59 Lexus
# 62 Land Rover
# 63 Kia
# 64 Daewoo
# 99 OTHER

## pozycja siedzenia pasażera

SEATPOS = 1

# 1: z przodu obok kierowcy 
# 2: drugi rząd
# 3: kolejny rząd 
# 4: inne miejsca (nieprzepisowe, bez siedzenia, na schodach, w korytarzu, na pace itp)

## wiek kierowcy z przedziału

DriverAge = 8

# 4: 17-21 lat
# 5: 22-30 lat
# 6: 31-50 lat
# 7: 51-65 lat
# 8: powyżej 66 lat

## pleć kierowcy 

DriverSex = 2

# 1: mężczyzna
# 2: kobieta


In [28]:
#2. kalkulator zbiorczego prawdopodobienstwa
## wiek auta
R1 = CarAgeResult.loc[CarAgeResult["CarAge"]== CarAge,["INJSEV","ratio"]].set_index("INJSEV")


## marka auta
R2 = MakeResult.loc[MakeResult["MAKE"]==MAKE,["INJSEV","ratio"]].set_index("INJSEV")

## pozycja siedzenia 
R3 = SeatposResult.loc[SeatposResult["SEATPOS"]==SEATPOS,["INJSEV","ratio"]].set_index("INJSEV")

## wiek kierowcy
R4 = DriverAgeResult.loc[DriverAgeResult["DriverAge"]==DriverAge,["INJSEV","ratio"]].set_index("INJSEV")

## pleć kierowcy
R5 = DriverSexResult.loc[DriverSexResult["DriverSex"]==DriverSex,["INJSEV","ratio"]].set_index("INJSEV")

In [29]:
#3. Wynik
wynik = (R1 + R2 + R3 + R4 + R5)/5
wynik

,ratio
INJSEV,
0,0.400286
1,0.374594
3,0.192393
4,0.032727


In [79]:
#4. Podsumowanie ogólne

report = CarAgeResult.merge(MakeResult, how = "outer", on = "INJSEV").merge(SeatposResult,how = "outer",on = "INJSEV").merge(DriverAgeResult,how = "outer",on = "INJSEV").merge(DriverSexResult,how = "outer",on = "INJSEV")

report.columns = ['CarAge', 'INJSEV', 'ratio_CarAge', 'MAKE', 'ratio_MAKE', 'SEATPOS', 'ratio_SEATPOS',
       'DriverAge', 'ratio_DriverAge', 'DriverSex', 'ratio_DriverSex']

report["TotalResult"] = (report["ratio_CarAge"] + report["ratio_MAKE"] + report["ratio_SEATPOS"] + report["ratio_DriverAge"] + report["ratio_DriverSex"])/5

report = report[report.columns[[1,0,3,5,7,9,11]]]

C:\Users\Dawid\anaconda3\lib\site-packages\pandas\core\frame.py:9186: FutureWarning: Passing 'suffixes' which cause duplicate columns {'ratio_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


In [80]:
# 10 kombinacji z najwyzszym prawdodopodobienstwem smierci
report.loc[report["INJSEV"]==4].sort_values("TotalResult", ascending = False).head(10)

,INJSEV,CarAge,MAKE,SEATPOS,DriverAge,DriverSex,TotalResult
28312,4,5,45.0,4.0,5.0,1.0,0.051281
28314,4,5,45.0,4.0,6.0,1.0,0.050884
27112,4,4,45.0,4.0,5.0,1.0,0.050056
28282,4,5,45.0,1.0,5.0,1.0,0.050026
27114,4,4,45.0,4.0,6.0,1.0,0.049660
28284,4,5,45.0,1.0,6.0,1.0,0.049629
28310,4,5,45.0,4.0,4.0,1.0,0.049510
28316,4,5,45.0,4.0,7.0,1.0,0.049403
27082,4,4,45.0,1.0,5.0,1.0,0.048801
25912,4,3,45.0,4.0,5.0,1.0,0.048413


In [82]:
# 10 kombinacji z najnizszym prawdodopodobienstwem smierci
report.loc[report["INJSEV"]==4].sort_values("TotalResult", ascending = True).head(10)

,INJSEV,CarAge,MAKE,SEATPOS,DriverAge,DriverSex,TotalResult
21949,4,0,30.0,3.0,8.0,2.0,0.022347
22589,4,0,55.0,3.0,8.0,2.0,0.022767
22629,4,0,59.0,3.0,8.0,2.0,0.023005
22549,4,0,53.0,3.0,8.0,2.0,0.023009
22669,4,0,62.0,3.0,8.0,2.0,0.023146
21947,4,0,30.0,3.0,7.0,2.0,0.023354
22709,4,0,63.0,3.0,8.0,2.0,0.023428
21669,4,0,6.0,3.0,8.0,2.0,0.023441
21941,4,0,30.0,3.0,4.0,2.0,0.023461
22469,4,0,51.0,3.0,8.0,2.0,0.023469


In [158]:
report.to_csv("dane_surowe_do_kalkulatora_task7.csv",index = False)